Задание No1.

Есть Pandas DataFrame со столбцами [“customer_id”, “product_id”, “timestamp”] , который содержит данные по просмотрам товаров на сайте. Есть проблема – просмотры одного customer_id не разбиты на сессии (появления на сайте). Мы хотим разместить сессии так, чтобы сессией считались все смежные просмотры, между которыми не более 3 минут.

Написать методом который создаст в Pandas DataFrame столбец session_id и проставит в нем уникальный int id для каждой сессии.

У каждого пользователя может быть по несколько сессий. Исходный DataFrame может быть большим – до 100 млн строк. 

In [1]:
import pandas as pd
import random
from datetime import datetime, timedelta

In [5]:
date_today = datetime.now()
ndays = [1,2,50,1,2,1,2,10,20,40,51,120]
df = pd.DataFrame({'customer_id': [1,1,1,2,2,3,3,3,3,3,1,1], 
                  'product_id': [1,2,2,3,4,5,6,7,8,9,9,9], 
                  'timestamp': [date_today + timedelta(minutes=x) for x in ndays]})
df

,customer_id,product_id,timestamp
0,1,1,2022-11-11 18:23:06.333164
1,1,2,2022-11-11 18:24:06.333164
2,1,2,2022-11-11 19:12:06.333164
3,2,3,2022-11-11 18:23:06.333164
4,2,4,2022-11-11 18:24:06.333164
5,3,5,2022-11-11 18:23:06.333164
6,3,6,2022-11-11 18:24:06.333164
7,3,7,2022-11-11 18:32:06.333164
8,3,8,2022-11-11 18:42:06.333164
9,3,9,2022-11-11 19:02:06.333164


In [6]:
def get_unique(col):
    for u in col.unique():
        col.replace(u, random.getrandbits(16), inplace=True)
    return col

In [7]:
df['session_id'] = (df.groupby('customer_id')['timestamp']
                   .transform(lambda col: get_unique(col.diff().gt('3Min').cumsum())))
df

,customer_id,product_id,timestamp,session_id
0,1,1,2022-11-11 18:23:06.333164,22814
1,1,2,2022-11-11 18:24:06.333164,22814
2,1,2,2022-11-11 19:12:06.333164,18508
3,2,3,2022-11-11 18:23:06.333164,51943
4,2,4,2022-11-11 18:24:06.333164,51943
5,3,5,2022-11-11 18:23:06.333164,30834
6,3,6,2022-11-11 18:24:06.333164,30834
7,3,7,2022-11-11 18:32:06.333164,16366
8,3,8,2022-11-11 18:42:06.333164,50712
9,3,9,2022-11-11 19:02:06.333164,61458


In [8]:
df.dtypes

customer_id             int64
product_id              int64
timestamp      datetime64[ns]
session_id              int32
dtype: object

In [9]:
new_df = df.copy()
new_df['session_id_notunique'] = (df.groupby('customer_id')['timestamp']
                   .transform(lambda col: col.diff().gt('3Min').cumsum()))
new_df

,customer_id,product_id,timestamp,session_id,session_id_notunique
0,1,1,2022-11-11 18:23:06.333164,22814,0
1,1,2,2022-11-11 18:24:06.333164,22814,0
2,1,2,2022-11-11 19:12:06.333164,18508,1
3,2,3,2022-11-11 18:23:06.333164,51943,0
4,2,4,2022-11-11 18:24:06.333164,51943,0
5,3,5,2022-11-11 18:23:06.333164,30834,0
6,3,6,2022-11-11 18:24:06.333164,30834,0
7,3,7,2022-11-11 18:32:06.333164,16366,1
8,3,8,2022-11-11 18:42:06.333164,50712,2
9,3,9,2022-11-11 19:02:06.333164,61458,3
